In [1]:
"""
Boilerplate to implement training different networks on different datasets
with varying config.

I wish a machine could automate setting up decent baseline models and datasets
"""
#  import json
import os
from os.path import exists
import pampy
from simple_parsing import ArgumentParser, choice
from simplepytorch import datasets as D
from simplepytorch import trainlib as TL
from simplepytorch import metrics
from sklearn.model_selection import StratifiedShuffleSplit
from torch.utils.data import Dataset, DataLoader
from typing import Union, Optional
import dataclasses as dc
import numpy as np
import torch as T
import torchvision.transforms as tvt
import torch
from torch import optim, nn
from torchvision import models, transforms

from deepfix.models import get_effnetv2, get_resnet, get_efficientnetv1, get_DeepFixEnd2End, DeepFixMLP
from deepfix.models.ghaarconv import convert_conv2d_to_gHaarConv2d
from deepfix.init_from_distribution import init_from_beta, reset_optimizer
from deepfix import deepfix_strategies as dfs
import pytorch_wavelets as pyw

In [2]:
import numpy as np
import scipy.io
# from ipynb.fs.defs.q2 import *
import sklearn
import torch
import torchvision.datasets
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import pandas as pd
# YOUR CODE HERE
# Use GPU if available, otherwise stick with cpu
use_cuda = torch.cuda.is_available()
torch.manual_seed(123)
device = torch.device("cuda" if use_cuda else "cpu")
print("device = {}".format(device))

device = cuda


In [3]:
MODELS = {
    ('effnetv2', str, str, str): (
        lambda pretrain, in_ch, out_ch: get_effnetv2(pretrain, int(in_ch), int(out_ch))),
    ('resnet50', str, str, str): (
        lambda pretrain, in_ch, out_ch: get_resnet('resnet50', pretrain, int(in_ch), int(out_ch))),
    ('resnet18', str, str, str): (
        lambda pretrain, in_ch, out_ch: get_resnet('resnet18', pretrain, int(in_ch), int(out_ch))),
    ('efficientnet-b0', str, str, str): (
        lambda pretrain, in_ch, out_ch: get_efficientnetv1('efficientnet-b0', pretrain, int(in_ch), int(out_ch))),
    ('efficientnet-b1', str, str, str): (
        lambda pretrain, in_ch, out_ch: get_efficientnetv1('efficientnet-b1', pretrain, int(in_ch), int(out_ch))),
    ('waveletres18', str, str, str): lambda pretrain, in_ch, out_ch: R(
        pretrain, int(in_ch), int(out_ch)),
    ('waveletmlp', str, str, str, str, str, str, str): (
        lambda mlp_channels, in_ch, out_ch, wavelet_levels, patch_size, in_ch_mul, mlp_depth: get_DeepFixEnd2End(
            int(in_ch), int(out_ch),
            in_ch_multiplier=int(in_ch_mul), wavelet='db1',
            wavelet_levels=int(wavelet_levels), wavelet_patch_size=int(patch_size),
            mlp_depth=int(mlp_depth), mlp_channels=int(mlp_channels),
            mlp_fix_weights='none', mlp_activation=None)
        ),

    #  ('waveletres18v2', str, str, str): lambda pretrain, in_ch, out_ch: (
        #  DeepFixCompression(levels=8, wavelet='coif1', patch_size=1),
        #  R2(pretrain, int(in_ch), int(out_ch))),
}


class R(T.nn.Module):
    def __init__(self, pretrain, in_ch, out_ch):
        super().__init__()
        self.r = get_resnet('resnet18', pretrain, in_ch, out_ch,)
        self.dwt = pyw.DWT(J=8, wave='coif1', mode='zero')

    @staticmethod
    def wavelet_coefficients_as_tensorimage(approx, detail, normalize=False):
        B,C = approx.shape[:2]
        fixed_dims = approx.shape[:-2] # num images in minibatch, num channels, etc
        output_shape = fixed_dims + (
            detail[0].shape[-2]*2,  # input img height
            detail[0].shape[-1]*2)  # input img width
        im = T.zeros(output_shape, device=approx.device, dtype=approx.dtype)
        if normalize:
            norm11 = lambda x: (x / max(x.min()*-1, x.max()))  # into [-1,+1] preserving sign
            #  approx = norm11(approx)
        im[..., :approx.shape[-2], :approx.shape[-1]] = approx if approx is not None else 0
        for level in detail:
            lh, hl, hh = level.unbind(-3)
            h,w = lh.shape[-2:]
            if normalize:
                lh, hl, hh = [norm11(x) for x in [lh, hl, hh]]
            #  im[:h, :w] = approx
            im[..., 0:h, w:w+w] = lh  # horizontal
            im[..., h:h+h, :w] = hl  # vertical
            im[..., h:h+h, w:w+w] = hh  # diagonal
        return im

    def forward(self, x):
        x = self.wavelet_coefficients_as_tensorimage(*self.dwt(x))
        return self.r(x)


class R2(T.nn.Module):
    def __init__(self, pretrain, in_ch, out_ch):
        super().__init__()
        self.r = get_resnet('resnet18', pretrain, in_ch, out_ch,)

    def forward(self, x):
        B,C,H = x.shape
        x = x.unsqueeze(-1).repeat(1,1,1,H)
        return self.r(x)


class LossCheXpertIdentity(T.nn.Module):
    def __init__(self, N):
        super().__init__()
        self.bce = T.nn.BCEWithLogitsLoss()
        self.N = N

    def forward(self, yhat, y):
        # absolute max possible num patients in chexpert is 223414
        # but let's just hash them into a smaller number of bins via modulo N
        assert self.N == yhat.shape[1], \
                f'note: model must have {self.N} binary predictions per sample'
        y_onehot = y.new_zeros(y.shape[0], self.N, dtype=T.float
                               ).scatter_(1, y.long()%self.N, 1)
        return self.bce(yhat[:, -1], y_onehot[:, -1])


class LossCheXpertUignore(T.nn.Module):
    def __init__(self):
        super().__init__()
        self.bce = T.nn.BCEWithLogitsLoss()

    def forward(self, yhat, y):
        ignore = (y != 2)  # ignore uncertainty labels
        return self.bce(yhat[ignore], y[ignore])


def loss_intelmobileodt(yhat, y):
    """BCE Loss with class balancing weights.

    Not sure this actually helps

    because Type 2 is the hardest class, it
    has the most samples, and it separates Type 1 from Type 3.  Arguably, Type 2
    samples are on the decision boundary between Type 1 and 3.
    Class balancing weights make it harder to focus on class 2.
    """
    #  assert y.shape == yhat.shape, 'sanity check'
    #  assert y.dtype == yhat.dtype, 'sanity check'

    # class distribution of stage='train'
    w = T.tensor([249, 781, 450], dtype=y.dtype, device=y.device)
    w = (w.max() / w).reshape(1, 3)
    # w can have any of the shapes:  (B,1) or (1,C) or (B,C)
    #  return T.nn.functional.binary_cross_entropy_with_logits(yhat, y, weight=w)
    return T.nn.functional.cross_entropy(yhat, y, weight=w)
    # can't apply focal loss unless do it manually.


def onehot(y, nclasses):
    return T.zeros((y.numel(), nclasses), dtype=y.dtype, device=y.device)\
            .scatter_(1, y.unsqueeze(1), 1)


def _upsample_pad_minibatch_imgs_to_same_size(batch, target_is_segmentation_mask=False):
    """a collate function for a dataloader of (x,y) samples.  """
    shapes = [item[0].shape for item in batch]
    H = max(h for c,h,w in shapes)
    W = max(w for c,h,w in shapes)
    X, Y = [], []
    for item in batch:
        h,w = item[0].shape[1:]
        dh, dw = (H-h), (W-w)
        padding = (dw//2, dw-dw//2, dh//2, dh-dh//2, )
        X.append(T.nn.functional.pad(item[0], padding))
        if target_is_segmentation_mask:
            Y.append(T.nn.functional.pad(item[1], padding))
        else:
            Y.append(item[1])
    return T.stack(X), T.stack(Y)


def get_dset_chexpert(train_frac=.8, val_frac=.2, small=False,
                      labels:str='diagnostic', num_identities=None):
    """
    Args:
        labels:  either "diagnostic" (the 14 classes defined as
            D.CheXpert.LABELS_DIAGNOSTIC) or "identity" ("patient", "study",
            "view", "index")
        small:  whether to use CheXpert_Small dataset (previously downsampled
            images) or the fully size dataset.
        num_identities:  used only if labels='identity'.  If
            num_identities=1000, then all patients get identified as coming
            from precisely 1 of 1000 bins.

    Returns:
        (
        {'train_dset': ..., 'val_dset': ..., 'test_dset': ...,
         'train_loader': ..., 'val_loader': ..., 'test_loader': ...
         },

        ('Pneumonia', 'Cardiomegaly', ...)  # class names defined by `labels`
        )
    """
    _label_cleanup_dct = dict(D.CheXpert.LABEL_CLEANUP_DICT)
    if labels == 'diagnostic':
        class_names = D.CheXpert.LABELS_DIAGNOSTIC
        for k in class_names:
            _label_cleanup_dct[k][np.nan] = 0  # remap missing value to negative
        get_ylabels = lambda dct: \
                D.CheXpert.format_labels(dct, labels=class_names).float()
    elif labels == 'identity':
        class_names = list(range(num_identities))
        get_ylabels = lambda dct: \
                (D.CheXpert.format_labels(dct, labels=['index']) % num_identities).long()
    else:
        raise NotImplementedError(f"unrecognized labels: {labels}")
    kws = dict(
        img_transform=tvt.Compose([
            #  tvt.RandomCrop((512, 512)),
            tvt.ToTensor(),  # full res 1024x1024 imgs
            tvt.Resize((224, 224)),
            tvt.Lambda(lambda x: x.repeat(3, 1, 1)),
            
        ]),
        getitem_transform=lambda dct: (dct['image'], get_ylabels(dct)),
        label_cleanup_dct=_label_cleanup_dct,
    )
    if small:
        kls = D.CheXpert_Small
    else:
        kls = D.CheXpert

    train_dset = kls(dataset_dir="../data/CheXpert-v1.0-small/",use_train_set=True, **kws)  #Edited Elvin

    N = len(train_dset)
    if train_frac + val_frac == 1:
        nsplits = [N - int(N*val_frac), int(N*val_frac), 0]
    else:
        a,b = int(N*train_frac), int(N*val_frac)
        nsplits = [a,b, N-a-b]
    train_dset, val_dset, _ = T.utils.data.random_split(train_dset, nsplits)
    test_dset = kls(dataset_dir="../data/CheXpert-v1.0-small/",use_train_set=False, **kws) #Edited Elvin
    batch_dct = dict(
        batch_size=1, collate_fn=_upsample_pad_minibatch_imgs_to_same_size,
        num_workers=int(os.environ.get("num_workers", 4)))  # upsample pad must take time
    train_loader=DataLoader(train_dset, shuffle=True, **batch_dct)
    val_loader=DataLoader(val_dset, **batch_dct)
    test_loader=DataLoader(test_dset, **batch_dct)
    return (dict(
        train_dset=train_dset, val_dset=val_dset, test_dset=test_dset,
        train_loader=train_loader, val_loader=val_loader, test_loader=test_loader,
    ), class_names)



def match(spec:str, dct:dict):
    return pampy.match(spec.split(':'), *(x for y in dct.items() for x in y))


def get_model_opt_loss(
        model_spec:str, opt_spec:str, loss_spec:str, regularizer_spec:str,
        device:str) -> dict[str, Union[T.nn.Module, T.optim.Optimizer]]:
    """
    Args:
        model_spec: a string of form,
            "model_name:pretraining:in_channels:out_classes".  For example:
            "effnetv2:untrained:1:5"
        opt_spec: Specifies how to create optimizer.
            First value is a pytorch Optimizer in T.optim.*.
            Other values are numerical parameters.
            Example: "SGD:lr=.003:momentum=.9"
        device: e.g. 'cpu' or 'gpu'
    Returns:
        a pytorch model and optimizer
    """
    mdl = match(model_spec, MODELS)
    mdl = mdl.to(device, non_blocking=True)
    optimizer = reset_optimizer(opt_spec, mdl)
    loss_fn = match(loss_spec, LOSS_FNS)
    if regularizer_spec != 'none':
        loss_fn = RegularizedLoss(mdl, loss_fn, regularizer_spec)
    return dict(model=mdl, optimizer=optimizer, loss_fn=loss_fn)


class RegularizedLoss(T.nn.Module):
    def __init__(self, model, lossfn, regularizer_spec:str):
        super().__init__()
        self.lossfn = lossfn
        self.regularizer_spec = regularizer_spec
        if regularizer_spec == 'none':
            self.regularizer = lambda *y: 0
        elif regularizer_spec.startswith('deepfixmlp:'):
            lbda = float(regularizer_spec.split(':')[1])
            self.regularizer = lambda *y: (
                float(lbda) * DeepFixMLP.get_VecAttn_regularizer(model))
        else:
            raise NotImplementedError(regularizer_spec)

    def forward(self, yhat, y):
        return self.lossfn(yhat, y) + self.regularizer(yhat, y)

    def __repr__(self):
        return f'RegularizedLoss<{repr(self.lossfn)},{self.regularizer_spec}>'


def get_dset_loaders_resultfactory(dset_spec:str) -> dict:
    dct, class_names = match(dset_spec, DSETS)
    if any(dset_spec.startswith(x) for x in {'intel_mobileodt:',
                                             'chexpert_small_ID:'}):
        #  dct['result_factory'] = lambda: TL.MultiLabelBinaryClassification(
                #  class_names, binarize_fn=lambda yh: (T.sigmoid(yh)>.5).long())
        dct['result_factory'] = lambda: TL.MultiClassClassification(
                len(class_names), binarize_fn=lambda yh: yh.softmax(1).argmax(1))
    elif any(dset_spec.startswith(x) for x in {'chexpert:', 'chexpert_small:'}):
        dct['result_factory'] = lambda: CheXpertMultiLabelBinaryClassification(
            class_names, binarize_fn=lambda yh: (yh.sigmoid()>.5).long(), report_avg=True)
    else:
        raise NotImplementedError(f"I don't know how to create the result factory for {dset_spec}")
    return dct

class CheXpertMultiLabelBinaryClassification(TL.MultiLabelBinaryClassification):
    def update(self, yhat, y, loss) -> None:
        self.num_samples += yhat.shape[0]
        self.loss += loss.item()
        assert yhat.shape == y.shape
        assert yhat.ndim == 2 and yhat.shape[1] == len(self._cms), "sanity check: model outputs expected prediction shape"
        binarized = self._binarize_fn(yhat)
        assert binarized.dtype == T.long, 'sanity check binarize fn'
        assert binarized.shape == y.shape, 'sanity check binarize fn'
        ignore = (y != 2)  # ignore uncertainty labels
        for i, (kls, cm) in enumerate(self._cms.items()):
            rows = ignore[:, i]
            if rows.sum() == 0:
                continue  # don't update a confusion matrix if all data for this class is ignored
            cm += metrics.confusion_matrix(y[rows, i], binarized[rows, i], num_classes=2).cpu()


def get_deepfix_train_strategy(args:'TrainOptions'):
    deepfix_spec = args.deepfix
    if deepfix_spec == 'off':
        return TL.train_one_epoch
    elif deepfix_spec.startswith('reinit:'):
        _, N, P, R = deepfix_spec.split(':')
        return dfs.DeepFix_TrainOneEpoch(int(N), float(P), int(R), TL.train_one_epoch)
    elif deepfix_spec.startswith('dhist:'):
        fp = deepfix_spec.split(':', 1)[1]
        assert exists(fp), f'histogram file not found: {fp}'
        return dfs.DeepFix_DHist(fp)
    elif deepfix_spec.startswith('dfhist:'):
        fp = deepfix_spec.split(':', 1)[1]
        assert exists(fp), f'histogram file not found: {fp}'
        return dfs.DeepFix_DHist(fp, fixed=True)
    elif deepfix_spec == 'fixed':
        return dfs.DeepFix_DHist('', fixed=True, init_with_hist=False)
    elif deepfix_spec.startswith('beta:'):
        alpha, beta = deepfix_spec.split(':')[1:]
        return dfs.DeepFix_LambdaInit(
            lambda cfg: init_from_beta(cfg.model, float(alpha), float(beta)))
    elif deepfix_spec.startswith('ghaarconv2d:'):
        ignore_layers = deepfix_spec.split(':')[1].split(',')
        return dfs.DeepFix_LambdaInit(
            lambda cfg: (
                print(f'initialize {deepfix_spec}'),
                convert_conv2d_to_gHaarConv2d(cfg.model, ignore_layers=ignore_layers),
                reset_optimizer(args.opt, cfg.model),
                print(cfg.model)
            ))
    else:
        raise NotImplementedError(deepfix_spec)


def train_config(args:'TrainOptions') -> TL.TrainConfig:
    return TL.TrainConfig(
        **get_model_opt_loss(
            args.model, args.opt, args.lossfn, args.loss_reg, args.device),
        **get_dset_loaders_resultfactory(args.dset),
        device=args.device,
        epochs=args.epochs,
        start_epoch=args.start_epoch,
        train_one_epoch=get_deepfix_train_strategy(args),
        experiment_id=args.experiment_id,
    )


@dc.dataclass
class TrainOptions:
    """High-level configuration for training PyTorch models
    on the IntelMobileODTCervical dataset.
    """
    epochs:int = 50
    start_epoch:int = 0  # if "--start_epoch 1", then don't evaluate perf before training.
    device:str = 'cuda' if T.cuda.is_available() else 'cpu'
    dset:str = None #choice(
        #  'intel_mobileodt:train:val:test:v1',
        #  'intel_mobileodt:train+additional:val:test:v1',
        #  'intel_mobileodt:train+additional:noval:test:v1',
        #  'chexpert:.8:.2', 'chexpert:.01:.01', 'chexpert:.001:.001',
        #  'chexpert_small:.8:.2', 'chexpert_small:.01:.01',
        #   'chexpert_small:.001:.001',
        #  default='intel_mobileodt:train:val:test:v1')
    opt:str = 'SGD:lr=.001:momentum=.9:nesterov=1'
    lossfn:str = None  # choices:
        #  'BCEWithLogitsLoss',
        #  'CrossEntropyLoss', 
        #  'CE_intelmobileodt',
        #  'chexpert_uignore', 
        #  'chexpert_identity:N' for some N=num_identities predicted by model (compared to identities y%N)
    loss_reg:str = 'none'  # Optionally add a regularizer to the loss.  loss + reg.  Accepted values:  'none', 'deepfixmlp:X' where X is a positive float denoting the lambda in l1 regularizer
    model:str = 'resnet18:imagenet:3:3'  # Model specification adheres to the template "model_name:pretraining:in_ch:out_ch"
    deepfix:str = 'off'  # DeepFix Re-initialization Method.
                         #  "off" or "reinit:N:P:R" or "d[f]hist:path_to_histogram.pth"
                         #  or "beta:A:B" for A,B as (float) parameters of the beta distribution
                         # 'ghaarconv2d:layer1,layer2' Replaces all spatial convolutions with GHaarConv2d layer except the specified layers
    experiment_id:str = os.environ.get('run_id', 'debugging')
    prune:str = 'off'

    def execute(self):
        cfg = train_config(self)
        cfg.train(cfg)


def main():
    p = ArgumentParser()
    p.add_arguments(TrainOptions, dest='TrainOptions')
#     for patch_size in [1,32]:
#         for wavelet_level in [1,2,3,4,5,6,7,8,9]:    
#             try:
    in_ch, out_ch = 3, 3
    model_params = "resnet18:imagenet:"+str(in_ch)+":"+str(in_ch)    
    
#     model_params = "waveletmlp:300:1:14:"+str(patch_size)+":"+str(wavelet_level)+":1:2"
    exp_id = 'model_'+model_params+'_in_ch_'+str(in_ch)+'out_ch_'+str(in_ch)#+'_patch_size_' + str(patch_size) + '_level_' + str(wavelet_level)
    args = p.parse_args(["--dset", "chexpert_small:.01:.01", "--opt", "Adam:lr=0.001", "--lossfn", "chexpert_uignore", "--model", model_params, "--loss_reg", "none","--experiment_id",exp_id]).TrainOptions

    print(args)
    cfg = train_config(args)

# python deepfix/train.py --dset chexpert_small:.01:.01 --opt Adam:lr=0.001 --lossfn chexpert_uignore --model waveletmlp:300:1:14:7:1:1:2 --loss_reg none    

    if args.prune != 'off':
        assert args.prune.startswith('ChannelPrune:')
        raise NotImplementedError('code is a bit hardcoded, so it is not available without hacking on it.')
        print(args.prune)
        from explainfix import channelprune
        from deepfix.weight_saliency import costfn_multiclass
        a = sum([x.numel() for x in cfg.model.parameters()])
        channelprune(cfg.model, pct=5, grad_cost_fn=costfn_multiclass,
                     loader=cfg.train_loader, device=cfg.device, num_minibatches=10)
        b = sum([x.numel() for x in cfg.model.parameters()])
        assert a/b != 1
        print(f'done channelpruning.  {a/b}')

    cfg.train(cfg)
#             except Exception as e:
#                 print("=================================================================================================")
#                 print(e)
#                 print("=================================================================================================")
            
    print('+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++')
    #  import IPython ; IPython.embed() ; import sys ; sys.exit()

    #  with T.profiler.profile(
    #      activities=[
    #          T.profiler.ProfilerActivity.CPU,
    #          T.profiler.ProfilerActivity.CUDA,
    #      ], with_modules=True,
    #  ) as p:
    #      cfg.train(cfg)
    #  print(p.key_averages().table(
    #      sort_by="self_cuda_time_total", row_limit=-1))


# if __name__ == "__main__":
#     main()


## Generate the Dataset

In [4]:
d,l = get_dset_chexpert()
d

{'train_dset': <torch.utils.data.dataset.Subset at 0x154056620250>,
 'val_dset': <torch.utils.data.dataset.Subset at 0x1540566204f0>,
 'test_dset': <simplepytorch.datasets.chexpert.CheXpert at 0x154056527d00>,
 'train_loader': <torch.utils.data.dataloader.DataLoader at 0x154055fe44c0>,
 'val_loader': <torch.utils.data.dataloader.DataLoader at 0x15404f1494f0>,
 'test_loader': <torch.utils.data.dataloader.DataLoader at 0x15404f149e50>}

In [5]:
trainset_loader = d['train_loader']
valset_loader = d['val_loader']

In [6]:
len(trainset_loader)

178732

In [7]:
model = models.resnet18(pretrained=True)
# model

In [9]:
new_model = torch.nn.Sequential(*list(model.children())[:-1])
device = torch.device('cuda:0' if torch.cuda.is_available() else "cpu")
new_model = new_model.to(device)

#### Generate the Embeddings

In [10]:
from tqdm import tqdm
import numpy as np

# Will contain the feature
X_train = []
Y_train = []
for data in tqdm(trainset_loader):
    inputs, labels = data
    inputs = inputs.to(device)
    labels = labels.to(device)
    with torch.no_grad():
        # Extract the feature from the image
        feature = new_model(inputs)
        X_train.append(feature.cpu().detach().numpy().reshape(512))
        Y_train.append(labels.cpu().detach().numpy()[0])
X_train = np.array(X_train)
Y_train = np.array(Y_train)

100%|██████████| 178732/178732 [29:43<00:00, 100.21it/s] 


In [11]:
# Will contain the feature
X_test = []
Y_test = []
for data in tqdm(valset_loader):
    inputs, labels = data
    inputs = inputs.to(device)
    labels = labels.to(device)
    with torch.no_grad():
        # Extract the feature from the image
        feature = new_model(inputs)
#         print(feature.size())
        # Convert to NumPy Array, Reshape it, and save it to features variable
        X_test.append(feature.cpu().detach().numpy().reshape(512))
        Y_test.append(labels.cpu().detach().numpy()[0])
#         features.append(feature.cpu().detach().numpy().reshape(-1,4096))
# Convert to NumPy Array
X_test = np.array(X_test)
Y_test = np.array(Y_test)

100%|██████████| 44682/44682 [06:32<00:00, 113.81it/s]


### Save Embeddings as CSV

In [12]:
pd.DataFrame(X_train).to_csv("feature_csv/resnet18_x_train.csv", index=False)  
pd.DataFrame(Y_train).to_csv("feature_csv/resnet18_y_train.csv", index=False)

pd.DataFrame(X_test).to_csv("feature_csv/resnet18_x_test.csv", index=False)  
pd.DataFrame(Y_test).to_csv("feature_csv/resnet18_y_test.csv", index=False)

### Read the saved CSV's

In [13]:
# pd.DataFrame(X_train).to_csv("feature_csv/vgg16_x_train.csv", index=False)  
# pd.DataFrame(Y_train).to_csv("feature_csv/vgg16_y_train.csv", index=False)
X_train = pd.read_csv("feature_csv/resnet18_x_train.csv")
Y_train = pd.read_csv("feature_csv/resnet18_y_train.csv")

# pd.DataFrame(X_test).to_csv("feature_csv/vgg16_x_test.csv", index=False)  
# pd.DataFrame(Y_test).to_csv("feature_csv/vgg16_y_test.csv", index=False)
X_test = pd.read_csv("feature_csv/resnet18_x_test.csv")
Y_test = pd.read_csv("feature_csv/resnet18_y_test.csv")

In [14]:
X_train = X_train.iloc[:,:].values
Y_train = Y_train.iloc[:,:].values

X_test = X_test.iloc[:,:].values
Y_test = Y_test.iloc[:,:].values

In [15]:
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
# from sklearn.preprocessing import StandardScaler
from sklearn.mixture import GaussianMixture
from sklearn.metrics import classification_report

In [16]:
# Y_train = Y_train.reshape(-1,)
# Y_test = Y_test.reshape(-1,)


In [17]:
Y_train.shape

(178732, 14)

### Training and Testing GMM

In [27]:
# pos_preds = []
# neg_preds = []

# pos_preds_probs = []
# neg_preds_probs = []

pos_scores = []
neg_scores = []

gmm_pos_models   = []
gmm_neg_models   = []

y_pred = np.zeros_like(Y_test)
for i in range(14):
    x_pos_train = X_train[Y_train[:, i] == 1]
    x_neg_train = X_train[Y_train[:, i] == 0]
    
    gmm_pos = GaussianMixture(n_components=3,init_params='random',covariance_type='diag',tol=0.001).fit(x_pos_train)
    print("model {} (pos) is trained".format(i) )
    
    gmm_neg = GaussianMixture(n_components=3,init_params='kmeans',covariance_type='diag',tol=0.001).fit(x_neg_train)
    print("model {} (neg) is trained".format(i) )

    pos_preds = gmm_pos.score_samples(X_test)
    neg_preds = gmm_neg.score_samples(X_test)
    
    for j in range(pos_preds.shape[0]):
        if pos_preds[j]>neg_preds[j]:
            y_pred[j,i] = 1

    gmm_pos_models.append(gmm_pos)
    gmm_neg_models.append(gmm_neg)

#     print("model {} is trained".format(i) )
print('Trained')

model 0 (pos) is trained
model 0 (neg) is trained
model 0 is trained
model 1 (pos) is trained
model 1 (neg) is trained
model 1 is trained
model 2 (pos) is trained
model 2 (neg) is trained
model 2 is trained
model 3 (pos) is trained
model 3 (neg) is trained
model 3 is trained
model 4 (pos) is trained
model 4 (neg) is trained
model 4 is trained
model 5 (pos) is trained
model 5 (neg) is trained
model 5 is trained
model 6 (pos) is trained
model 6 (neg) is trained
model 6 is trained
model 7 (pos) is trained
model 7 (neg) is trained
model 7 is trained
model 8 (pos) is trained
model 8 (neg) is trained
model 8 is trained
model 9 (pos) is trained
model 9 (neg) is trained
model 9 is trained
model 10 (pos) is trained
model 10 (neg) is trained
model 10 is trained
model 11 (pos) is trained
model 11 (neg) is trained
model 11 is trained
model 12 (pos) is trained
model 12 (neg) is trained
model 12 is trained
model 13 (pos) is trained
model 13 (neg) is trained
model 13 is trained


In [26]:
# 3 componenets, Spherical Covariance
for i in range(Y_test.shape[1]):
    acc = sum(y_pred[:,i]*Y_test[:,i])
    print('Accuracy for class {}: {}'.format(i,acc/Y_test.shape[0]))

Accuracy for class 0: 0.05677901615863211
Accuracy for class 1: 0.07085627321964102
Accuracy for class 2: 0.10274831028154514
Accuracy for class 3: 0.29188487534130075
Accuracy for class 4: 0.02837831789087328
Accuracy for class 5: 0.2129269056890918
Accuracy for class 6: 0.15435745937961595
Accuracy for class 7: 0.08186741864733002
Accuracy for class 8: 0.2554272413947451
Accuracy for class 9: 0.06369455261626605
Accuracy for class 10: 0.274786267400743
Accuracy for class 11: 0.02141802067946824
Accuracy for class 12: 0.022581800277516672
Accuracy for class 13: 0.337496083434045


In [ ]:
# pos_preds = []
# neg_preds = []

# pos_preds_probs = []
# neg_preds_probs = []

pos_scores = []
neg_scores = []

gmm_pos_models   = []
gmm_neg_models   = []

y_pred = np.zeros_like(Y_test)
for i in range(14):
    x_pos_train = X_train[Y_train[:, i] == 1]
    x_neg_train = X_train[Y_train[:, i] == 0]
    
    gmm_pos = GaussianMixture(n_components=3,init_params='random',covariance_type='spherical',tol=0.001).fit(x_pos_train)
    print("model {} (pos) is trained".format(i) )
    
    gmm_neg = GaussianMixture(n_components=3,init_params='kmeans',covariance_type='spherical',tol=0.001).fit(x_neg_train)
    print("model {} (neg) is trained".format(i) )

    pos_preds = gmm_pos.score_samples(X_test)
    neg_preds = gmm_neg.score_samples(X_test)
    
    for j in range(pos_preds.shape[0]):
        if pos_preds[j]>neg_preds[j]:
            y_pred[j,i] = 1

    gmm_pos_models.append(gmm_pos)
    gmm_neg_models.append(gmm_neg)

#     print("model {} is trained".format(i) )
print('Trained')

In [28]:
# 3 componenets, Diagonal Covariance
for i in range(Y_test.shape[1]):
    acc = sum(y_pred[:,i]*Y_test[:,i])
    print('Accuracy for class {}: {}'.format(i,acc/Y_test.shape[0]))

Accuracy for class 0: 0.05812183877176492
Accuracy for class 1: 0.06783492234009221
Accuracy for class 2: 0.10004028467839399
Accuracy for class 3: 0.2964504722259523
Accuracy for class 4: 0.027259299046595943
Accuracy for class 5: 0.2167539501365203
Accuracy for class 6: 0.1495904391029945
Accuracy for class 7: 0.08012174925025738
Accuracy for class 8: 0.24931739850499082
Accuracy for class 9: 0.061187950405084825
Accuracy for class 10: 0.27809856317980397
Accuracy for class 11: 0.019940915805022157
Accuracy for class 12: 0.022805604046372142
Accuracy for class 13: 0.34163645315787117


In [29]:
# pos_preds = []
# neg_preds = []

# pos_preds_probs = []
# neg_preds_probs = []

pos_scores = []
neg_scores = []

gmm_pos_models   = []
gmm_neg_models   = []

y_pred = np.zeros_like(Y_test)
for i in range(14):
    x_pos_train = X_train[Y_train[:, i] == 1]
    x_neg_train = X_train[Y_train[:, i] == 0]
    
    gmm_pos = GaussianMixture(n_components=4,init_params='random',covariance_type='spherical',tol=0.001).fit(x_pos_train)
    print("model {} (pos) is trained".format(i) )
    
    gmm_neg = GaussianMixture(n_components=4,init_params='kmeans',covariance_type='spherical',tol=0.001).fit(x_neg_train)
    print("model {} (neg) is trained".format(i) )

    pos_preds = gmm_pos.score_samples(X_test)
    neg_preds = gmm_neg.score_samples(X_test)
    
    for j in range(pos_preds.shape[0]):
        if pos_preds[j]>neg_preds[j]:
            y_pred[j,i] = 1

    gmm_pos_models.append(gmm_pos)
    gmm_neg_models.append(gmm_neg)

#     print("model {} is trained".format(i) )
print('Trained')

model 0 (pos) is trained
model 0 (neg) is trained
model 1 (pos) is trained
model 1 (neg) is trained
model 2 (pos) is trained
model 2 (neg) is trained
model 3 (pos) is trained
model 3 (neg) is trained
model 4 (pos) is trained
model 4 (neg) is trained
model 5 (pos) is trained
model 5 (neg) is trained
model 6 (pos) is trained
model 6 (neg) is trained
model 7 (pos) is trained
model 7 (neg) is trained
model 8 (pos) is trained
model 8 (neg) is trained
model 9 (pos) is trained
model 9 (neg) is trained
model 10 (pos) is trained
model 10 (neg) is trained
model 11 (pos) is trained
model 11 (neg) is trained
model 12 (pos) is trained
model 12 (neg) is trained
model 13 (pos) is trained
model 13 (neg) is trained
Trained


In [30]:
# 4 componenets, spherical Covariance
for i in range(Y_test.shape[1]):
    acc = sum(y_pred[:,i]*Y_test[:,i])
    print('Accuracy for class {}: {}'.format(i,acc/Y_test.shape[0]))

Accuracy for class 0: 0.05886039120898796
Accuracy for class 1: 0.07018486191307462
Accuracy for class 2: 0.10015218656282172
Accuracy for class 3: 0.29347388210017455
Accuracy for class 4: 0.028691643167270936
Accuracy for class 5: 0.22152097041314175
Accuracy for class 6: 0.1514703907613804
Accuracy for class 7: 0.07985318472763081
Accuracy for class 8: 0.24766125061546038
Accuracy for class 9: 0.06253077301821763
Accuracy for class 10: 0.27330916252629694
Accuracy for class 11: 0.021261358041269416
Accuracy for class 12: 0.023186070453426435
Accuracy for class 13: 0.3344971129313818


In [31]:
# pos_preds = []
# neg_preds = []

# pos_preds_probs = []
# neg_preds_probs = []

pos_scores = []
neg_scores = []

gmm_pos_models   = []
gmm_neg_models   = []

y_pred = np.zeros_like(Y_test)
for i in range(14):
    x_pos_train = X_train[Y_train[:, i] == 1]
    x_neg_train = X_train[Y_train[:, i] == 0]

    
    gmm_pos = GaussianMixture(n_components=4,init_params='random',covariance_type='diag',tol=0.001).fit(x_pos_train)
    print("model {} (pos) is trained".format(i) )
    
    gmm_neg = GaussianMixture(n_components=4,init_params='kmeans',covariance_type='diag',tol=0.001).fit(x_neg_train)
    print("model {} (neg) is trained".format(i) )

    pos_preds = gmm_pos.score_samples(X_test)
    neg_preds = gmm_neg.score_samples(X_test)
    
    for j in range(pos_preds.shape[0]):
        if pos_preds[j]>neg_preds[j]:
            y_pred[j,i] = 1

    gmm_pos_models.append(gmm_pos)
    gmm_neg_models.append(gmm_neg)

#     print("model {} is trained".format(i) )
print('Trained')

model 0 (pos) is trained
model 0 (neg) is trained
model 1 (pos) is trained
model 1 (neg) is trained
model 2 (pos) is trained
model 2 (neg) is trained
model 3 (pos) is trained
model 3 (neg) is trained
model 4 (pos) is trained
model 4 (neg) is trained
model 5 (pos) is trained
model 5 (neg) is trained
model 6 (pos) is trained
model 6 (neg) is trained
model 7 (pos) is trained
model 7 (neg) is trained
model 8 (pos) is trained
model 8 (neg) is trained
model 9 (pos) is trained
model 9 (neg) is trained
model 10 (pos) is trained
model 10 (neg) is trained
model 11 (pos) is trained
model 11 (neg) is trained
model 12 (pos) is trained
model 12 (neg) is trained
model 13 (pos) is trained
model 13 (neg) is trained
Trained


In [32]:
# 4 componenets, Diagonal Covariance
for i in range(Y_test.shape[1]):
    acc = sum(y_pred[:,i]*Y_test[:,i])
    print('Accuracy for class {}: {}'.format(i,acc/Y_test.shape[0]))

Accuracy for class 0: 0.05950942213866881
Accuracy for class 1: 0.07172910791817734
Accuracy for class 2: 0.10035360995479164
Accuracy for class 3: 0.2920862987332707
Accuracy for class 4: 0.02486459871984244
Accuracy for class 5: 0.21731345955865897
Accuracy for class 6: 0.14600957880130702
Accuracy for class 7: 0.07358667919967772
Accuracy for class 8: 0.24358802202229085
Accuracy for class 9: 0.06042701759097623
Accuracy for class 10: 0.27662145830535784
Accuracy for class 11: 0.018351909046148338
Accuracy for class 12: 0.02141802067946824
Accuracy for class 13: 0.33740656192650287


In [33]:
# pos_preds = []
# neg_preds = []

# pos_preds_probs = []
# neg_preds_probs = []

pos_scores = []
neg_scores = []

gmm_pos_models   = []
gmm_neg_models   = []

y_pred = np.zeros_like(Y_test)
for i in range(14):
    x_pos_train = X_train[Y_train[:, i] == 1]
    x_neg_train = X_train[Y_train[:, i] == 0]

    gmm_pos = GaussianMixture(n_components=2,init_params='random',covariance_type='diag',tol=0.001).fit(x_pos_train)
    print("model {} (pos) is trained".format(i) )
    
    gmm_neg = GaussianMixture(n_components=2,init_params='kmeans',covariance_type='diag',tol=0.001).fit(x_neg_train)
    print("model {} (neg) is trained".format(i) )

    pos_preds = gmm_pos.score_samples(X_test)
    neg_preds = gmm_neg.score_samples(X_test)
    
    for j in range(pos_preds.shape[0]):
        if pos_preds[j]>neg_preds[j]:
            y_pred[j,i] = 1

    gmm_pos_models.append(gmm_pos)
    gmm_neg_models.append(gmm_neg)

#     print("model {} is trained".format(i) )
print('Trained')

model 0 (pos) is trained
model 0 (neg) is trained
model 1 (pos) is trained
model 1 (neg) is trained
model 2 (pos) is trained
model 2 (neg) is trained
model 3 (pos) is trained
model 3 (neg) is trained
model 4 (pos) is trained
model 4 (neg) is trained
model 5 (pos) is trained
model 5 (neg) is trained
model 6 (pos) is trained
model 6 (neg) is trained
model 7 (pos) is trained
model 7 (neg) is trained
model 8 (pos) is trained
model 8 (neg) is trained
model 9 (pos) is trained
model 9 (neg) is trained
model 10 (pos) is trained
model 10 (neg) is trained
model 11 (pos) is trained
model 11 (neg) is trained
model 12 (pos) is trained
model 12 (neg) is trained
model 13 (pos) is trained
model 13 (neg) is trained
Trained


In [34]:
# 2 componenets, Diagonal Covariance
for i in range(Y_test.shape[1]):
    acc = sum(y_pred[:,i]*Y_test[:,i])
    print('Accuracy for class {}: {}'.format(i,acc/Y_test.shape[0]))

Accuracy for class 0: 0.061568416812139114
Accuracy for class 1: 0.0661563940736762
Accuracy for class 2: 0.0974665413365561
Accuracy for class 3: 0.2832012891097086
Accuracy for class 4: 0.028512600152186564
Accuracy for class 5: 0.20592184772391567
Accuracy for class 6: 0.1478223893290363
Accuracy for class 7: 0.0816659952553601
Accuracy for class 8: 0.24580367933396
Accuracy for class 9: 0.062217447741819976
Accuracy for class 10: 0.271675395013652
Accuracy for class 11: 0.0206794682422452
Accuracy for class 12: 0.023767960252450652
Accuracy for class 13: 0.33315429031824895


In [35]:
# pos_preds = []
# neg_preds = []

# pos_preds_probs = []
# neg_preds_probs = []

pos_scores = []
neg_scores = []

gmm_pos_models   = []
gmm_neg_models   = []

y_pred = np.zeros_like(Y_test)
for i in range(14):
    x_pos_train = X_train[Y_train[:, i] == 1]
    x_neg_train = X_train[Y_train[:, i] == 0]
    
    gmm_pos = GaussianMixture(n_components=2,init_params='random',covariance_type='spherical',tol=0.001).fit(x_pos_train)
    print("model {} (pos) is trained".format(i) )
    
    gmm_neg = GaussianMixture(n_components=2,init_params='kmeans',covariance_type='spherical',tol=0.001).fit(x_neg_train)
    print("model {} (neg) is trained".format(i) )

    pos_preds = gmm_pos.score_samples(X_test)
    neg_preds = gmm_neg.score_samples(X_test)
    
    for j in range(pos_preds.shape[0]):
        if pos_preds[j]>neg_preds[j]:
            y_pred[j,i] = 1

    gmm_pos_models.append(gmm_pos)
    gmm_neg_models.append(gmm_neg)

#     print("model {} is trained".format(i) )
print('Trained')

model 0 (pos) is trained
model 0 (neg) is trained
model 1 (pos) is trained
model 1 (neg) is trained
model 2 (pos) is trained
model 2 (neg) is trained
model 3 (pos) is trained
model 3 (neg) is trained
model 4 (pos) is trained
model 4 (neg) is trained
model 5 (pos) is trained
model 5 (neg) is trained
model 6 (pos) is trained
model 6 (neg) is trained
model 7 (pos) is trained
model 7 (neg) is trained
model 8 (pos) is trained
model 8 (neg) is trained
model 9 (pos) is trained
model 9 (neg) is trained
model 10 (pos) is trained
model 10 (neg) is trained
model 11 (pos) is trained
model 11 (neg) is trained
model 12 (pos) is trained
model 12 (neg) is trained
model 13 (pos) is trained
model 13 (neg) is trained
Trained


In [36]:
# 2 componenets, spherical Covariance
for i in range(Y_test.shape[1]):
    acc = sum(y_pred[:,i]*Y_test[:,i])
    print('Accuracy for class {}: {}'.format(i,acc/Y_test.shape[0]))

Accuracy for class 0: 0.06235173000313325
Accuracy for class 1: 0.07016248153618906
Accuracy for class 2: 0.09997314354773734
Accuracy for class 3: 0.1778792354863256
Accuracy for class 4: 0.0296316189964639
Accuracy for class 5: 0.20283335571371022
Accuracy for class 6: 0.15191799829909136
Accuracy for class 7: 0.08882771585873506
Accuracy for class 8: 0.25157781657043105
Accuracy for class 9: 0.06396311713889262
Accuracy for class 10: 0.2695716395864106
Accuracy for class 11: 0.021888008594064725
Accuracy for class 12: 0.024707936081643615
Accuracy for class 13: 0.3304462647150978


In [37]:
# pos_preds = []
# neg_preds = []

# pos_preds_probs = []
# neg_preds_probs = []

pos_scores = []
neg_scores = []

gmm_pos_models   = []
gmm_neg_models   = []

y_pred = np.zeros_like(Y_test)
for i in range(14):
    x_pos_train = X_train[Y_train[:, i] == 1]
    x_neg_train = X_train[Y_train[:, i] == 0]
    
    gmm_pos = GaussianMixture(n_components=3,init_params='random',covariance_type='spherical',tol=0.001).fit(x_pos_train)
    print("model {} (pos) is trained".format(i) )
    
    gmm_neg = GaussianMixture(n_components=6,init_params='kmeans',covariance_type='spherical',tol=0.001).fit(x_neg_train)
    print("model {} (neg) is trained".format(i) )

    pos_preds = gmm_pos.score_samples(X_test)
    neg_preds = gmm_neg.score_samples(X_test)
    
    for j in range(pos_preds.shape[0]):
        if pos_preds[j]>neg_preds[j]:
            y_pred[j,i] = 1

    gmm_pos_models.append(gmm_pos)
    gmm_neg_models.append(gmm_neg)

#     print("model {} is trained".format(i) )
print('Trained')

model 0 (pos) is trained
model 0 (neg) is trained
model 1 (pos) is trained
model 1 (neg) is trained
model 2 (pos) is trained
model 2 (neg) is trained
model 3 (pos) is trained
model 3 (neg) is trained
model 4 (pos) is trained
model 4 (neg) is trained
model 5 (pos) is trained
model 5 (neg) is trained
model 6 (pos) is trained
model 6 (neg) is trained
model 7 (pos) is trained
model 7 (neg) is trained
model 8 (pos) is trained
model 8 (neg) is trained
model 9 (pos) is trained
model 9 (neg) is trained
model 10 (pos) is trained
model 10 (neg) is trained
model 11 (pos) is trained
model 11 (neg) is trained
model 12 (pos) is trained
model 12 (neg) is trained
model 13 (pos) is trained
model 13 (neg) is trained
Trained


In [38]:
# 2 componenets, spherical Covariance
for i in range(Y_test.shape[1]):
    acc = sum(y_pred[:,i]*Y_test[:,i])
    print('Accuracy for class {}: {}'.format(i,acc/Y_test.shape[0]))

Accuracy for class 0: 0.04626023902242514
Accuracy for class 1: 0.05841278367127702
Accuracy for class 2: 0.07571281500380467
Accuracy for class 3: 0.21892484669441833
Accuracy for class 4: 0.01839666979991943
Accuracy for class 5: 0.1719484356116557
Accuracy for class 6: 0.11986929859898841
Accuracy for class 7: 0.06532832012891097
Accuracy for class 8: 0.18407859988362205
Accuracy for class 9: 0.04661832505259388
Accuracy for class 10: 0.21207645136744102
Accuracy for class 11: 0.015106754397744058
Accuracy for class 12: 0.016516718141533503
Accuracy for class 13: 0.2825298778031422


In [39]:
# pos_preds = []
# neg_preds = []

# pos_preds_probs = []
# neg_preds_probs = []

pos_scores = []
neg_scores = []

gmm_pos_models   = []
gmm_neg_models   = []

y_pred = np.zeros_like(Y_test)
for i in range(14):
    x_pos_train = X_train[Y_train[:, i] == 1]
    x_neg_train = X_train[Y_train[:, i] == 0]
    
    gmm_pos = GaussianMixture(n_components=3,init_params='random',covariance_type='spherical',tol=0.001).fit(x_pos_train)
    print("model {} (pos) is trained".format(i) )
    
    gmm_neg = GaussianMixture(n_components=6,init_params='kmeans',covariance_type='spherical',tol=0.001).fit(x_neg_train)
    print("model {} (neg) is trained".format(i) )

    pos_preds = gmm_pos.score_samples(X_test)
    neg_preds = gmm_neg.score_samples(X_test)
    
    for j in range(pos_preds.shape[0]):
        if pos_preds[j]>neg_preds[j]:
            y_pred[j,i] = 1

    gmm_pos_models.append(gmm_pos)
    gmm_neg_models.append(gmm_neg)

#     print("model {} is trained".format(i) )
print('Trained')

model 0 (pos) is trained
model 0 (neg) is trained
model 1 (pos) is trained
model 1 (neg) is trained
model 2 (pos) is trained
model 2 (neg) is trained
model 3 (pos) is trained
model 3 (neg) is trained
model 4 (pos) is trained
model 4 (neg) is trained
model 5 (pos) is trained
model 5 (neg) is trained
model 6 (pos) is trained
model 6 (neg) is trained
model 7 (pos) is trained
model 7 (neg) is trained
model 8 (pos) is trained
model 8 (neg) is trained
model 9 (pos) is trained
model 9 (neg) is trained
model 10 (pos) is trained
model 10 (neg) is trained
model 11 (pos) is trained
model 11 (neg) is trained
model 12 (pos) is trained
model 12 (neg) is trained
model 13 (pos) is trained
model 13 (neg) is trained
Trained


In [40]:
# 2 componenets, spherical Covariance
for i in range(Y_test.shape[1]):
    acc = sum(y_pred[:,i]*Y_test[:,i])
    print('Accuracy for class {}: {}'.format(i,acc/Y_test.shape[0]))

Accuracy for class 0: 0.04305984512779195
Accuracy for class 1: 0.058144219148650464
Accuracy for class 2: 0.07522044671232264
Accuracy for class 3: 0.22861554988586008
Accuracy for class 4: 0.020164719573877624
Accuracy for class 5: 0.1716127299583725
Accuracy for class 6: 0.11962311445324739
Accuracy for class 7: 0.06503737522939887
Accuracy for class 8: 0.18392193724542322
Accuracy for class 9: 0.046819748444563804
Accuracy for class 10: 0.21180788684481447
Accuracy for class 11: 0.015218656282171792
Accuracy for class 12: 0.01653909851841905
Accuracy for class 13: 0.25795622398281187
